# 1. Energy data

## 1.0 Startup

In [1]:
disp ('executing gpml startup script...')
mydir = fileparts (mfilename ('fullpath'));                 % where am I located
addpath (mydir)
dirs = {'cov','doc','inf','lik','mean','prior','util'};           % core folders
for d = dirs, addpath (fullfile (mydir, d{1})), end
dirs = {{'util','minfunc'},{'util','minfunc','compiled'}};     % minfunc folders
for d = dirs, addpath (fullfile (mydir, d{1}{:})), end
% addpath([mydir,'/util/sparseinv'])
pkg load statistics

executing gpml startup script...


In [2]:
dataset = 'energy_data'
load ('./data/energy_data.mat');
[ndata, ~] = size(M);
X = M(:,[1:25,27:28]);
Y = M(:,26);
R = randperm(ndata);        
xt = X(R(1:1735),:);
yt = Y(R(1:1735));
R(1:1735) = [];
x = X(R,:);          
y = Y(R);
n = 18000; nt = 1735;   %Ms = 15

dataset = energy_data


In [3]:
sf2 = 1 ; ell = 1 ; sn2 = 0.1 ; 
d = size(x,2);
hyp.cov = log([ones(d,1)*ell;sf2]); hyp.lik = log(sn2); hyp.mean = [];
opts.Xnorm = 'Y' ; opts.Ynorm = 'Y' ;
opts.Ms = 50 ;
opts.ell = ell ; opts.sf2 = sf2 ; opts.sn2 = sn2 ;
opts.meanfunc = []; opts.covfunc = @covSEard; opts.likfunc = @likGauss; opts.inffunc = @infGaussLik ;

partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
[models,t_dGP_train] = aggregation_train(x,y,opts) ;
save('./res/energymodel', 'models','t_dGP_train')

In [4]:
partitionCriterion = 'kmeans'; %, 'kmeans', 'knkmeans'
opts.numOptFC = 25 ;
opts.partitionCriterion = partitionCriterion ;
s = load('./res/energymodel');
models = s.models;


In [5]:
criterions = {'BCM','PoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans BCM
BCM: SMSE 0.00171094, MSLL -3.2778, NLPD -0.3996
Processing energy_data kmeans PoE
PoE: SMSE 0.00209438, MSLL -2.8853, NLPD -0.0071


## 1.1 RBCM vs TERBCM

### 1.1.1 RBCM baseline

In [6]:
criterions = {'RBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans RBCM
RBCM: SMSE 0.00141779, MSLL 2.7120, NLPD 5.5902


### 1.1.2 Grid q

In [7]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TERBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for i = 1:length(partitionCriterions)
    partitionCriterion = partitionCriterions{i};
    for j = 1:length(criterions)     
        criterion = criterions{j};
        printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
        opts.partitionCriterion = partitionCriterion ;
        %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
        for q = qs
            [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
            [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
            printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
            filename = sprintf('./res/terbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
            save(filename, 'SMSE','MSLL','NLPD')
        endfor
    endfor
endfor


Processing energy_data kmeans TERBCM
TERBCM: SMSE 0.00160261, MSLL 4.4335, NLPD 7.3117 - q   0.20
TERBCM: SMSE 0.00156991, MSLL 2.0795, NLPD 4.9577 - q   0.40
TERBCM: SMSE 0.00152777, MSLL 1.5166, NLPD 4.3948 - q   0.60
TERBCM: SMSE 0.00147640, MSLL 1.7570, NLPD 4.6352 - q   0.80
TERBCM: SMSE 0.00141779, MSLL 2.7120, NLPD 5.5902 - q   1.00
TERBCM: SMSE 0.00135548, MSLL 4.5921, NLPD 7.4703 - q   1.20
TERBCM: SMSE 0.00129346, MSLL 7.8932, NLPD 10.7714 - q   1.40
TERBCM: SMSE 0.00123504, MSLL 13.5452, NLPD 16.4234 - q   1.60
TERBCM: SMSE 0.00118213, MSLL 23.2355, NLPD 26.1137 - q   1.80
TERBCM: SMSE 0.00113536, MSLL 40.0311, NLPD 42.9093 - q   2.00


### 1.1.3 Find single q

In [8]:
criterion = 'TERBCM';

In [9]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.6);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.6000
grad_q_norm =  129.97
grad_q_reg = -60.828
Optimizing q: remain iter - 999, q - 0.6012
grad_q_norm =  110.88
grad_q_reg = -92.450
Optimizing q: remain iter - 998, q - 0.6031
grad_q_norm =  62.363
grad_q_reg = -74.883
Optimizing q: remain iter - 997, q - 0.6047
grad_q_norm =  75.812
grad_q_reg = -105.53
Optimizing q: remain iter - 996, q - 0.6068
grad_q_norm =  59.213
grad_q_reg = -100.45
Optimizing q: remain iter - 995, q - 0.6089
grad_q_norm =  90.482
grad_q_reg = -101.37
O

grad_q_norm =  1.1784
grad_q_reg = -9.1063
Optimizing q: remain iter - 911, q - 0.6989
grad_q_norm =  40.226
grad_q_reg = -56.224
Optimizing q: remain iter - 910, q - 0.7001
grad_q_norm =  7.7173
grad_q_reg = -14.026
Optimizing q: remain iter - 909, q - 0.7004
grad_q_norm =  10.131
grad_q_reg = -14.689
Optimizing q: remain iter - 908, q - 0.7007
grad_q_norm =  16.915
grad_q_reg = -31.827
Optimizing q: remain iter - 907, q - 0.7013
grad_q_norm =  30.165
grad_q_reg = -36.064
Optimizing q: remain iter - 906, q - 0.7020
grad_q_norm =  80.610
grad_q_reg = -47.729
Optimizing q: remain iter - 905, q - 0.7030
grad_q_norm =  0.47204
grad_q_reg = -16.824
Optimizing q: remain iter - 904, q - 0.7034
grad_q_norm =  39.898
grad_q_reg = -46.837
Optimizing q: remain iter - 903, q - 0.7043
grad_q_norm = -1.1263
grad_q_reg = -7.3294
Optimizing q: remain iter - 902, q - 0.7045
grad_q_norm =  48.632
grad_q_reg = -49.405
Optimizing q: remain iter - 901, q - 0.7055
grad_q_norm =  32.697
grad_q_reg = -46.744

grad_q_norm = -19.808
grad_q_reg =  12.180
Optimizing q: remain iter - 817, q - 0.7442
grad_q_norm = -28.269
grad_q_reg =  13.926
Optimizing q: remain iter - 816, q - 0.7439
grad_q_norm =  45.393
grad_q_reg = -27.522
Optimizing q: remain iter - 815, q - 0.7445
grad_q_norm =  27.223
grad_q_reg = -29.709
Optimizing q: remain iter - 814, q - 0.7451
grad_q_norm =  36.157
grad_q_reg = -39.515
Optimizing q: remain iter - 813, q - 0.7459
grad_q_norm =  12.085
grad_q_reg = -26.767
Optimizing q: remain iter - 812, q - 0.7464
grad_q_norm = -6.5444
grad_q_reg = -8.3661
Optimizing q: remain iter - 811, q - 0.7466
grad_q_norm =  18.575
grad_q_reg = -31.789
Optimizing q: remain iter - 810, q - 0.7473
grad_q_norm = -8.4154
grad_q_reg = -3.2125
Optimizing q: remain iter - 809, q - 0.7473
grad_q_norm =  36.311
grad_q_reg = -30.425
Optimizing q: remain iter - 808, q - 0.7479
grad_q_norm =  39.822
grad_q_reg = -50.782
Optimizing q: remain iter - 807, q - 0.7490
grad_q_norm =  14.872
grad_q_reg = -29.371


grad_q_norm =  13.274
grad_q_reg = -27.561
Optimizing q: remain iter - 723, q - 0.7639
grad_q_norm = -14.834
grad_q_reg =  3.8817
Optimizing q: remain iter - 722, q - 0.7638
grad_q_norm =  2.9672
grad_q_reg = -11.504
Optimizing q: remain iter - 721, q - 0.7641
grad_q_norm = -23.190
grad_q_reg =  6.1759
Optimizing q: remain iter - 720, q - 0.7639
grad_q_norm =  19.416
grad_q_reg = -24.380
Optimizing q: remain iter - 719, q - 0.7644
grad_q_norm =  19.055
grad_q_reg = -32.160
Optimizing q: remain iter - 718, q - 0.7651
grad_q_norm =  13.146
grad_q_reg = -24.468
Optimizing q: remain iter - 717, q - 0.7656
grad_q_norm = -39.716
grad_q_reg =  29.957
Optimizing q: remain iter - 716, q - 0.7650
grad_q_norm =  53.060
grad_q_reg = -18.993
Optimizing q: remain iter - 715, q - 0.7654
grad_q_norm =  22.804
grad_q_reg = -29.790
Optimizing q: remain iter - 714, q - 0.7660
grad_q_norm = -19.733
grad_q_reg =  6.2372
Optimizing q: remain iter - 713, q - 0.7659
grad_q_norm =  25.367
grad_q_reg = -38.935


grad_q_norm = -5.9830
grad_q_reg =  0.12925
Optimizing q: remain iter - 629, q - 0.7769
grad_q_norm =  5.8497
grad_q_reg = -7.8957
Optimizing q: remain iter - 628, q - 0.7771
grad_q_norm = -5.1155
grad_q_reg = -9.0585
Optimizing q: remain iter - 627, q - 0.7773
grad_q_norm = -1.3348
grad_q_reg = -6.9331


...


grad_q_norm = -46.756
grad_q_reg =  32.494
Optimizing q: remain iter - 451, q - 0.7801
grad_q_norm =  44.519
grad_q_reg = -29.430
Optimizing q: remain iter - 450, q - 0.7807
grad_q_norm = -42.503
grad_q_reg =  33.435
Optimizing q: remain iter - 449, q - 0.7800
grad_q_norm =  39.277
grad_q_reg = -47.145
Optimizing q: remain iter - 448, q - 0.7810
grad_q_norm =  29.072
grad_q_reg = -22.865
Optimizing q: remain iter - 447, q - 0.7814
grad_q_norm = -66.945
grad_q_reg =  56.430
Optimizing q: remain iter - 446, q - 0.7803
grad_q_norm =  59.703
grad_q_reg = -23.014
Optimizing q: remain iter - 445, q - 0.7808
grad_q_norm = -26.850
grad_q_reg =  10.461
Optimizing q: remain iter - 444, q - 0.7805
grad_q_norm = -8.3133
grad_q_reg = -5.6707
Optimizing q: remain iter - 443, q - 0.7807
grad_q_norm = -2.4823
grad_q_reg = -5.3279
Optimizing q: remain iter - 442, q - 0.7808
grad_q_norm = -16.462
grad_q_reg =  8.2113
Optimizing q: remain iter - 441, q - 0.7806
grad_q_norm = -46.471
grad_q_reg =  33.924

grad_q_norm = -2.7046
grad_q_reg =  0.52536
Optimizing q: remain iter - 357, q - 0.7800
grad_q_norm = -2.6233
grad_q_reg = -9.3875
Optimizing q: remain iter - 356, q - 0.7802
grad_q_norm = -3.8984
grad_q_reg = -6.4238
Optimizing q: remain iter - 355, q - 0.7803
grad_q_norm =  54.864
grad_q_reg = -18.317
Optimizing q: remain iter - 354, q - 0.7807
grad_q_norm = -9.3483
grad_q_reg = -6.4778
Optimizing q: remain iter - 353, q - 0.7808
grad_q_norm = -50.223
grad_q_reg =  36.985
Optimizing q: remain iter - 352, q - 0.7801
grad_q_norm =  8.2247
grad_q_reg = -9.2752
Optimizing q: remain iter - 351, q - 0.7803
grad_q_norm =  32.457
grad_q_reg = -22.650
Optimizing q: remain iter - 350, q - 0.7807
grad_q_norm = -46.799
grad_q_reg =  38.017
Optimizing q: remain iter - 349, q - 0.7799
grad_q_norm =  1.9174
grad_q_reg =  3.0558
Optimizing q: remain iter - 348, q - 0.7799
grad_q_norm =  2.8821
grad_q_reg = -2.0591
Optimizing q: remain iter - 347, q - 0.7799
grad_q_norm = -21.551
grad_q_reg =  5.1443

grad_q_norm =  46.172
grad_q_reg = -26.106
Optimizing q: remain iter - 263, q - 0.7844
grad_q_norm =  13.770
grad_q_reg = -11.912
Optimizing q: remain iter - 262, q - 0.7846
grad_q_norm =  3.4745
grad_q_reg = -12.040
Optimizing q: remain iter - 261, q - 0.7849
grad_q_norm =  25.602
grad_q_reg = -21.052
Optimizing q: remain iter - 260, q - 0.7853
grad_q_norm = -49.171
grad_q_reg =  34.539
Optimizing q: remain iter - 259, q - 0.7846
grad_q_norm = -13.456
grad_q_reg =  6.6743
Optimizing q: remain iter - 258, q - 0.7845
grad_q_norm =  24.081
grad_q_reg = -14.450
Optimizing q: remain iter - 257, q - 0.7848
grad_q_norm =  0.13213
grad_q_reg = -15.323
Optimizing q: remain iter - 256, q - 0.7851
grad_q_norm =  22.172
grad_q_reg = -16.161
Optimizing q: remain iter - 255, q - 0.7854
grad_q_norm =  7.5791
grad_q_reg = -17.787
Optimizing q: remain iter - 254, q - 0.7858
grad_q_norm = -48.142
grad_q_reg =  33.356
Optimizing q: remain iter - 253, q - 0.7851
grad_q_norm = -1.2220
grad_q_reg = -2.6590

grad_q_norm =  30.317
grad_q_reg = -31.625
Optimizing q: remain iter - 169, q - 0.7829
grad_q_norm = -53.913
grad_q_reg =  44.206
Optimizing q: remain iter - 168, q - 0.7820
grad_q_norm = -3.5655
grad_q_reg = -4.2689
Optimizing q: remain iter - 167, q - 0.7820
grad_q_norm =  2.0370
grad_q_reg =  9.3527
Optimizing q: remain iter - 166, q - 0.7819
grad_q_norm = -4.9278
grad_q_reg = -1.0939
Optimizing q: remain iter - 165, q - 0.7819
grad_q_norm = -26.513
grad_q_reg =  11.467
Optimizing q: remain iter - 164, q - 0.7816
grad_q_norm = -9.1611
grad_q_reg =  6.1921
Optimizing q: remain iter - 163, q - 0.7815
grad_q_norm =  32.419
grad_q_reg = -41.101
Optimizing q: remain iter - 162, q - 0.7824
grad_q_norm = -61.652
grad_q_reg =  46.001
Optimizing q: remain iter - 161, q - 0.7814
grad_q_norm = -23.697
grad_q_reg =  27.756
Optimizing q: remain iter - 160, q - 0.7808
grad_q_norm =  19.921
grad_q_reg = -22.866
Optimizing q: remain iter - 159, q - 0.7813
grad_q_norm = -43.225
grad_q_reg =  37.443


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [10]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.1.4 Find multi q

In [11]:
criterion = 'TERBCM';

In [12]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1)
qs = mul_grad_q(models, criterion, opts, iter=1000, lr=1e-7, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
iqs =

   0.79225
   0.78922
   0.78369
   0.78397
   0.78909
   0.78354
   0.78921
   0.78643
   0.78429
   0.79262
   0.78576
   0.78890
   0.78710
   0.79008
   0.78424
   0.78957
   0.78943
   0.78663
   0.78911
   0.79130
   0.79019
   0.79062
   0.79275
   0.79193
   0.78858
   0.78824
   0.78813
   0.78800
   0.78842
   0.79283
   0.79099
   0.79109
   0.78767
   0.78863
   0.78829
   0.78770
   0.78396
   0.78915
   0.79007
   0.79038
   0.79113
   0.78691
   0.78443
   0.78757
   0.78487
   0.79189
   0.78619
   0.78731
   0.78399
   0.78950

Start preprocessing...
Finish preprocessing...
Optimizing qs: remain iter - 1000
qs =

   0.79225
   0.78922
   0.78369
   0.78397
   0.78909
   0.78354
   0.78921
   0.78643
   0.78429
   0.79262
   0.78576
   0.78890
   0.78710
   0.79008
   0.78424
   0.78957
   0.78943
   0.78663
   0.78911
   0.79130
   0.79019
   0.79062
   0.79275
   0.79193
   0.78858
   0.78824
   0.78813
   0.78800
   0.78842
   0.79283
   0.79099
 

   0.79079
   0.78799
   0.79001
   0.79207
   0.79131
   0.78932
   0.78823
   0.78920
   0.78758
   0.78672
   0.78714
   0.79381
   0.79146
   0.78892
   0.78756
   0.78988
   0.79028
   0.79025
   0.78368
   0.79111
   0.78755
   0.78933
   0.79134
   0.78731
   0.78565
   0.78702
   0.78557
   0.79424
   0.78652
   0.78892
   0.78584
   0.78740

qs_dist =  0.0033135
Optimizing qs: remain iter - 987
qs =

   0.77859
   0.78840
   0.78627
   0.78326
   0.78840
   0.78574
   0.78791
   0.78367
   0.78255
   0.79219
   0.78592
   0.78838
   0.78788
   0.79146
   0.78474
   0.79091
   0.78912
   0.78651
   0.79097
   0.78809
   0.79030
   0.79195
   0.78977
   0.78958
   0.78852
   0.78925
   0.78778
   0.78689
   0.78734
   0.79397
   0.79166
   0.78921
   0.78786
   0.79016
   0.79057
   0.79031
   0.78397
   0.78992
   0.78785
   0.78928
   0.79162
   0.78731
   0.78594
   0.78732
   0.78584
   0.79453
   0.78663
   0.78922
   0.78612
   0.78769

qs_dist =  0.0040094
Optimizing qs: 

   0.79039
   0.78807
   0.78769
   0.78340
   0.78565
   0.79714
   0.78507
   0.79142
   0.78822
   0.78540

qs_dist =  0.0031891
Optimizing qs: remain iter - 974
qs =

   0.76522
   0.78812
   0.78905
   0.78147
   0.79157
   0.78921
   0.78469
   0.78020
   0.78289
   0.79173
   0.78208
   0.78800
   0.78882
   0.79102
   0.78541
   0.79178
   0.78899
   0.78755
   0.79046
   0.78159
   0.78968
   0.79459
   0.78923
   0.78617
   0.78531
   0.79041
   0.78613
   0.78755
   0.78676
   0.79512
   0.79187
   0.78452
   0.78863
   0.79222
   0.79004
   0.79388
   0.78208
   0.79228

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.78926
   0.79077
   0.78087
   0.79463
   0.79211
   0.78174
   0.77593
   0.78364
   0.78981
   0.78102
   0.78969
   0.78820
   0.79129
   0.78587
   0.79452
   0.79011
   0.78742
   0.79160
   0.77575
   0.78937
   0.79725
   0.78862
   0.78228
   0.78274
   0.79221
   0.78498
   0.78725
   0.78704
   0.79704
   0.78926
   0.78164
   0.78670
   0.79355
   0.79059
   0.79696
   0.78011
   0.79471
   0.78203
   0.78313
   0.78948
   0.79104
   0.78934
   0.78358
   0.78659
   0.80031
   0.78656
   0.79200
   0.78756
   0.78324

qs_dist =  0.0027031
Optimizing qs: remain iter - 958
qs =

   0.75252
   0.78920
   0.79097
   0.78117
   0.79486
   0.79241
   0.78181
   0.77559
   0.78378
   0.79001
   0.78098
   0.78998
   0.78820
   0.79095
   0.78587
   0.79477
   0.79040
   0.78744
   0.79189
   0.77579
   0.78949
   0.79752
   0.78870
   0.78083
   0.78272
   0.79245
   0.78528
   0.78753
   0.78665
   0.79733
   0.78941
   0.78164
   0.78699
   0.79364
   0.79079
   0.79725
   

...

   0.76962
   0.77498
   0.76532
   0.81820
   0.81819
   0.80484
   0.81119
   0.84785
   0.78298
   0.80302
   0.79897
   0.71161
   0.78177
   0.84676
   0.76066
   0.72899
   0.75571
   0.83658
   0.77114
   0.80519
   0.82445
   0.84524
   0.78055
   0.74157
   0.79440
   0.83917
   0.84092
   0.88629
   0.74770
   0.80126
   0.73711
   0.74731
   0.79199
   0.80568
   0.83590
   0.78498
   0.80118
   0.89425
   0.80779
   0.85623
   0.84469
   0.76736

qs_dist =  0.0031832
Optimizing qs: remain iter - 305
qs =

   0.60529
   0.81447
   0.85956
   0.76756
   0.82487
   0.86635
   0.73873
   0.70953
   0.76988
   0.77515
   0.76502
   0.81848
   0.81820
   0.80495
   0.81119
   0.84805
   0.78327
   0.80325
   0.79926
   0.71174
   0.78203
   0.84696
   0.76088
   0.72899
   0.75588
   0.83683
   0.77137
   0.80547
   0.82264
   0.84542
   0.78086
   0.74098
   0.79468
   0.83926
   0.84076
   0.88650
   0.74740
   0.80154
   0.73723
   0.74761
   0.79223
   0.80596
   0.83598
   0

   0.84102
   0.76809
   0.80445
   0.82119
   0.84898
   0.78251
   0.74343
   0.79731
   0.84334
   0.84177
   0.89355
   0.74797
   0.80090
   0.73731
   0.73986
   0.79092
   0.80557
   0.83986
   0.78553
   0.80617
   0.90292
   0.80790
   0.86276
   0.85332
   0.76802

qs_dist =  0.0015915
Optimizing qs: remain iter - 237
qs =

   0.60008
   0.81670
   0.86506
   0.76010
   0.82438
   0.87153
   0.73614
   0.71045
   0.76619
   0.77294
   0.76255
   0.81918
   0.82203
   0.80848
   0.81352
   0.84810
   0.78724
   0.80690
   0.79964
   0.70654
   0.78384
   0.84900
   0.75529
   0.72520
   0.75634
   0.84102
   0.76836
   0.80460
   0.82130
   0.84898
   0.78242
   0.74369
   0.79734
   0.84336
   0.84201
   0.89373
   0.74803
   0.80110
   0.73733
   0.73988
   0.79099
   0.80574
   0.83987
   0.78569
   0.80618
   0.90303
   0.80762
   0.86278
   0.85355
   0.76832

qs_dist =  0.0031989
Optimizing qs: remain iter - 236
qs =

   0.60027
   0.81694
   0.86520
   0.76043
   0.8245

   0.87929
   0.73307
   0.70583
   0.76414
   0.77183
   0.76120
   0.82069
   0.82395
   0.80933
   0.81517
   0.84727
   0.78553
   0.80779
   0.80022
   0.70622
   0.77770
   0.85330
   0.75447
   0.71936
   0.75588
   0.84369
   0.76917
   0.80837
   0.82444
   0.85085
   0.77880
   0.74106
   0.80249
   0.84999
   0.84546
   0.90081
   0.74319
   0.80266
   0.73466
   0.73760
   0.79084
   0.80683
   0.84453
   0.78136
   0.80427
   0.91116
   0.80577
   0.86842
   0.85616
   0.76860

qs_dist =  0.0016640
Optimizing qs: remain iter - 172
qs =

   0.60028
   0.81561
   0.87033
   0.76216
   0.82831
   0.87885
   0.73338
   0.70601
   0.76397
   0.77212
   0.76146
   0.82025
   0.82395
   0.80936
   0.81517
   0.84721
   0.78575
   0.80785
   0.80046
   0.70650
   0.77741
   0.85303
   0.75421
   0.71970
   0.75612
   0.84370
   0.76946
   0.80857
   0.82459
   0.85088
   0.77899
   0.74137
   0.80258
   0.85004
   0.84569
   0.90081
   0.74347
   0.80252
   0.73487
   0.73788
   0

   0.84711
   0.77917
   0.80769
   0.91393
   0.80712
   0.87143
   0.86018
   0.76708

qs_dist =  0.0027964
Optimizing qs: remain iter - 143
qs =

   0.60038
   0.81400
   0.87431
   0.76442
   0.82953
   0.88053
   0.72952
   0.70481
   0.76371
   0.77121
   0.76138
   0.82132
   0.82532
   0.81025
   0.81586
   0.84885
   0.78817
   0.80879
   0.79835
   0.70302
   0.77785
   0.85390
   0.75306
   0.71870
   0.75542
   0.84283
   0.77066
   0.81069
   0.82634
   0.85291
   0.77941
   0.73983
   0.80601
   0.84870
   0.84600
   0.90236
   0.74080
   0.80171
   0.73423
   0.73576
   0.79154
   0.80654
   0.84731
   0.77948
   0.80788
   0.91411
   0.80631
   0.87164
   0.86040
   0.76729

qs_dist =  0.0022956
Optimizing qs: remain iter - 142
qs =

   0.60038
   0.81424
   0.87439
   0.76474
   0.82970
   0.88073
   0.72939
   0.70478
   0.76402
   0.77143
   0.76149
   0.82147
   0.82532
   0.81041
   0.81586
   0.84904
   0.78766
   0.80884
   0.79858
   0.70305
   0.77801
   0.8541

   0.76504
   0.76988
   0.75962
   0.82082
   0.82660
   0.81096
   0.81718
   0.84821
   0.78774
   0.80868
   0.79957
   0.70369
   0.77814
   0.85270
   0.75428
   0.72083
   0.75632
   0.84289
   0.76880
   0.80985
   0.82764
   0.85183
   0.77898
   0.74166
   0.80390
   0.85000
   0.84720
   0.90414
   0.74049
   0.80252
   0.73323
   0.73713
   0.78807
   0.80659
   0.84660
   0.77790
   0.80355
   0.91444
   0.80769
   0.87058
   0.86102
   0.76939

qs_dist =  0.0034263
Optimizing qs: remain iter - 129
qs =

   0.59972
   0.81597
   0.87462
   0.76583
   0.82874
   0.88056
   0.73120
   0.70596
   0.76380
   0.77004
   0.75987
   0.82033
   0.82663
   0.81105
   0.81719
   0.84833
   0.78797
   0.80888
   0.79969
   0.70384
   0.77801
   0.85202
   0.75362
   0.72108
   0.75659
   0.84295
   0.76901
   0.81002
   0.82787
   0.85201
   0.77928
   0.74198
   0.80418
   0.85020
   0.84744
   0.90374
   0.74081
   0.80252
   0.73356
   0.73706
   0.78836
   0.80493
   0.84681
   0

In [13]:
qs
save('./res/terbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.59994
   0.81919
   0.87754
   0.76304
   0.83501
   0.88880
   0.73022
   0.70500
   0.76261
   0.77206
   0.76318
   0.82007
   0.83112
   0.81291
   0.81839
   0.85474
   0.78955
   0.80789
   0.80799
   0.70220
   0.77906
   0.85787
   0.75210
   0.72243
   0.75998
   0.85158
   0.76960
   0.81027
   0.83604
   0.85916
   0.77856
   0.73935
   0.80094
   0.84896
   0.85190
   0.91548
   0.73880
   0.79971
   0.73281
   0.73778
   0.79055
   0.80993
   0.84550
   0.77899
   0.80565
   0.92646
   0.81186
   0.87565
   0.87144
   0.76816

TERBCM: SMSE 0.00147045, MSLL 1.6471, NLPD 4.5253


## 1.2 GRBCM vs TEGRBCM

### 1.2.1 GRBCM baseline

In [14]:
criterions = {'GRBCM'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GRBCM
GRBCM: SMSE 0.00154576, MSLL -4.3647, NLPD -1.4865


### 1.2.2 Grid q

In [15]:
criterions = {'TEGRBCM'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegrbcm%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGRBCM
TEGRBCM: SMSE 0.00236596, MSLL -4.4640, NLPD -1.5858 - q   0.20
TEGRBCM: SMSE 0.00215229, MSLL -4.5088, NLPD -1.6306 - q   0.40
TEGRBCM: SMSE 0.00189737, MSLL -4.5522, NLPD -1.6740 - q   0.60
TEGRBCM: SMSE 0.00168440, MSLL -4.5370, NLPD -1.6588 - q   0.80
TEGRBCM: SMSE 0.00154576, MSLL -4.3647, NLPD -1.4865 - q   1.00
TEGRBCM: SMSE 0.00149046, MSLL -3.8500, NLPD -0.9718 - q   1.20
TEGRBCM: SMSE 0.00150596, MSLL -2.6423, NLPD 0.2359 - q   1.40
TEGRBCM: SMSE 0.00156091, MSLL -0.0745, NLPD 2.8037 - q   1.60
TEGRBCM: SMSE 0.00162070, MSLL 5.1377, NLPD 8.0159 - q   1.80
TEGRBCM: SMSE 0.00166452, MSLL 15.4973, NLPD 18.3755 - q   2.00


### 1.2.3 Find single q

In [29]:
criterion = 'TEGRBCM';

In [30]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.6);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.6000
grad_q_norm = -2.2321
grad_q_reg =  0.87640
Optimizing q: remain iter - 999, q - 0.6000
grad_q_norm = -0.0032946
grad_q_reg =    1.0642e-04
Optimizing q: remain iter - 998, q - 0.6000
grad_q_norm = -1.1151
grad_q_reg =  0.098005
Optimizing q: remain iter - 997, q - 0.6000
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 996, q - 0.6000
Optimizing q: remain iter - 996, q - 0.6000
grad_q_norm =  0.17920
grad_q_reg =  0.047427
Optimizing q: remain iter - 995, q - 0.60

grad_q_norm = -0.83876
grad_q_reg =  0.28228
Optimizing q: remain iter - 914, q - 0.5994
grad_q_norm =  0.13948
grad_q_reg =  0.46698
Optimizing q: remain iter - 913, q - 0.5994
grad_q_norm = -1.7953
grad_q_reg =  0.33287
Optimizing q: remain iter - 912, q - 0.5994
grad_q_norm = -0.58445
grad_q_reg =  0.15963
Optimizing q: remain iter - 911, q - 0.5994
grad_q_norm = -1.0683
grad_q_reg =  0.34697
Optimizing q: remain iter - 910, q - 0.5994
grad_q_norm =  2.0150
grad_q_reg =  1.4392
Optimizing q: remain iter - 909, q - 0.5994
grad_q_norm = -0.14269
grad_q_reg =  0.0047305
Optimizing q: remain iter - 908, q - 0.5994
grad_q_norm = -1.1108
grad_q_reg =  0.65415
Optimizing q: remain iter - 907, q - 0.5993
grad_q_norm =  0.18540
grad_q_reg =  0.33690
Optimizing q: remain iter - 906, q - 0.5993
grad_q_norm = -0.41310
grad_q_reg =  0.58250
Optimizing q: remain iter - 905, q - 0.5993
grad_q_norm = -5.2209
grad_q_reg =  0.71437
Optimizing q: remain iter - 904, q - 0.5993
grad_q_norm =  0.056585
g

Optimizing q: remain iter - 822, q - 0.5988
grad_q_norm = -2.0062
grad_q_reg =  1.3504
Optimizing q: remain iter - 821, q - 0.5987
grad_q_norm = -1.1209
grad_q_reg =  0.28731
Optimizing q: remain iter - 820, q - 0.5987
grad_q_norm =  0.18823
grad_q_reg =  0.069154
Optimizing q: remain iter - 819, q - 0.5987
grad_q_norm =  0.22114
grad_q_reg =  0.12735
Optimizing q: remain iter - 818, q - 0.5987
grad_q_norm = -1.9351
grad_q_reg =  0.88685
Optimizing q: remain iter - 817, q - 0.5987
Optimizing q: remain iter - 817, q - 0.5987
grad_q_norm =  3.3541
grad_q_reg =  0.64059
Optimizing q: remain iter - 816, q - 0.5987
grad_q_norm =  0.073634
grad_q_reg =  0.11349
Optimizing q: remain iter - 815, q - 0.5987
grad_q_norm =  0.41633
grad_q_reg =  0.12981
Optimizing q: remain iter - 814, q - 0.5987
grad_q_norm =  0.12336
grad_q_reg =  0.15704
Optimizing q: remain iter - 813, q - 0.5987
grad_q_norm = -5.7282
grad_q_reg =  1.7872
Optimizing q: remain iter - 812, q - 0.5986
grad_q_norm = -2.1193
grad_

grad_q_reg = 0
Optimizing q: remain iter - 731, q - 0.5981
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 730, q - 0.5981
grad_q_norm = -0.72209
grad_q_reg =  0.27246
Optimizing q: remain iter - 729, q - 0.5981
grad_q_norm =  0.57860
grad_q_reg =  0.16230
Optimizing q: remain iter - 728, q - 0.5981
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 727, q - 0.5981
grad_q_norm = -1.1184
grad_q_reg =  0.65758
Optimizing q: remain iter - 726, q - 0.5981
grad_q_norm = -2.0275
grad_q_reg =  0.52537
Optimizing q: remain iter - 725, q - 0.5980
grad_q_norm = -1.1756
grad_q_reg =  0.54916
Optimizing q: remain iter - 724, q - 0.5980
grad_q_norm = -1.4340
grad_q_reg =  0.69728
Optimizing q: remain iter - 723, q - 0.5980
grad_q_norm = -0.15731
grad_q_reg =  1.2586
Optimizing q: remain iter - 722, q - 0.5980
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 721, q - 0.5980
grad_q_norm =  0.17294
grad_q_reg =  0.082924
Optimizing q: remain iter - 720, q - 0.5980
grad_q_n

Optimizing q: remain iter - 639, q - 0.5976
grad_q_norm =    1.7272e-15
grad_q_reg = 0
Optimizing q: remain iter - 638, q - 0.5976
grad_q_norm = -0.26635
grad_q_reg =  0.60279
Optimizing q: remain iter - 637, q - 0.5976
grad_q_norm = -1.0416
grad_q_reg =  0.47763
Optimizing q: remain iter - 636, q - 0.5976
grad_q_norm = -0.11672
grad_q_reg =  0.010936
Optimizing q: remain iter - 635, q - 0.5976
grad_q_norm = -0.64543
grad_q_reg =  0.018124
Optimizing q: remain iter - 634, q - 0.5976
grad_q_norm =  0.10945
grad_q_reg =  0.0064710
Optimizing q: remain iter - 633, q - 0.5976
grad_q_norm =  0.099792
grad_q_reg =  0.20062


...

Optimizing q: remain iter - 306, q - 0.5953
grad_q_norm =  0.70647
grad_q_reg =  0.53098
Optimizing q: remain iter - 305, q - 0.5953
grad_q_norm = -0.19315
grad_q_reg =  0.15092
Optimizing q: remain iter - 304, q - 0.5953
grad_q_norm = -0.82453
grad_q_reg =  0.040261
Optimizing q: remain iter - 303, q - 0.5953
grad_q_norm = -0.84125
grad_q_reg =  0.46544
Optimizing q: remain iter - 302, q - 0.5953
Optimizing q: remain iter - 302, q - 0.5953
grad_q_norm =  0.65269
grad_q_reg =  1.0120
Optimizing q: remain iter - 301, q - 0.5952
grad_q_norm =  0.41784
grad_q_reg =  0.41533
Optimizing q: remain iter - 300, q - 0.5952
grad_q_norm = -0.21224
grad_q_reg =  0.026642
Optimizing q: remain iter - 299, q - 0.5952
grad_q_norm = -0.69510
grad_q_reg =  0.19083
Optimizing q: remain iter - 298, q - 0.5952
grad_q_norm =  3.1090
grad_q_reg =  0.97092
Optimizing q: remain iter - 297, q - 0.5952
grad_q_norm = -0.79395
grad_q_reg =  0.59427
Optimizing q: remain iter - 296, q - 0.5952
grad_q_norm =    1.397

Optimizing q: remain iter - 214, q - 0.5948
grad_q_norm = -1.4273
grad_q_reg =  0.39560
Optimizing q: remain iter - 213, q - 0.5948
grad_q_norm = -2.6870
grad_q_reg =  0.81073
Optimizing q: remain iter - 212, q - 0.5948
grad_q_norm =  0.40605
grad_q_reg =  0.10837
Optimizing q: remain iter - 211, q - 0.5948
grad_q_norm =  0.21522
grad_q_reg =  0.35715
Optimizing q: remain iter - 210, q - 0.5947
grad_q_norm = -1.3977
grad_q_reg =  0.74578
Optimizing q: remain iter - 209, q - 0.5947
grad_q_norm =    4.7258e-16
grad_q_reg = 0
Optimizing q: remain iter - 208, q - 0.5947
grad_q_norm =   -2.6949e-15
grad_q_reg = 0
Optimizing q: remain iter - 207, q - 0.5947
grad_q_norm =  0.16779
grad_q_reg =  0.50239
Optimizing q: remain iter - 206, q - 0.5947
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 205, q - 0.5947
grad_q_norm =  0.025814
grad_q_reg =  0.0011902
Optimizing q: remain iter - 204, q - 0.5947
grad_q_norm = -1.5118
grad_q_reg =  0.13810
Optimizing q: remain iter - 203, q - 0.5

Optimizing q: remain iter - 123, q - 0.5942
grad_q_norm = -0.62192
grad_q_reg =  0.10348
Optimizing q: remain iter - 122, q - 0.5942
grad_q_norm = -0.17749
grad_q_reg =  0.13483
Optimizing q: remain iter - 121, q - 0.5942
grad_q_norm = 0
grad_q_reg = 0
Optimizing q: remain iter - 120, q - 0.5942
grad_q_norm =  0.38635

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




grad_q_reg =  0.37642
Optimizing q: remain iter - 36, q - 0.5936
grad_q_norm =  0.98338
grad_q_reg =  0.22915
Optimizing q: remain iter - 35, q - 0.5936
grad_q_norm =  0.95756
grad_q_reg =  0.059884
Optimizing q: remain iter - 34, q - 0.5936
grad_q_norm =  0.032040
grad_q_reg =  2.4708
Optimizing q: remain iter - 33, q - 0.5936
grad_q_norm =  0.074763
grad_q_reg =  0.14121
Optimizing q: remain iter - 32, q - 0.5936
grad_q_norm =  0.12946
grad_q_reg =  0.15973
Optimizing q: remain iter - 31, q - 0.5936
grad_q_norm =  0.38688
grad_q_reg =  0.18320
Optimizing q: remain iter - 30, q - 0.5936
grad_q_norm = -0.088673
grad_q_reg =  0.27462
Optimizing q: remain iter - 29, q - 0.5936
grad_q_norm =  0.041568
grad_q_reg =  0.063028
Optimizing q: remain iter - 28, q - 0.5936
grad_q_norm = -0.23140
grad_q_reg =  0.19095
Optimizing q: remain iter - 27, q - 0.5935
grad_q_norm =  0.60077
grad_q_reg =  0.050710
Optimizing q: remain iter - 26, q - 0.5935
grad_q_norm = -1.3028
grad_q_reg =  0.79997
Opti

In [31]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.2.4 Find multi q

In [32]:
criterion = 'TEGRBCM';

In [37]:
printf('Optimizing q\r\n');
iqs = q + 0.01 * rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=5000, lr=1e-8, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing qs: remain iter - 5000
qs =

   0.60024
   0.59522
   0.59854
   0.59591
   0.59456
   0.59789
   0.60022
   0.59742
   0.60273
   0.60118
   0.60162
   0.59950
   0.59861
   0.60278
   0.60200
   0.60059
   0.60313
   0.60042
   0.60003
   0.59588
   0.59521
   0.60195
   0.59447
   0.60318
   0.59552
   0.59962
   0.59755
   0.60270
   0.60264
   0.59518
   0.59618
   0.60242
   0.59423
   0.59432
   0.59521
   0.59973
   0.59680
   0.60065
   0.59577
   0.59703
   0.60113
   0.59374
   0.59776
   0.59370
   0.59374
   0.60131
   0.60235
   0.59564
   0.60001
   0.60020

qs_dist =    5.3602e-09
Optimizing qs: remain iter - 4999
qs =

   0.60024
   0.59522
   0.59854
   0.59591
   0.59456
   0.59789
   0.60022
   0.59742
   0.60273
   0.60118
   0.60162
   0.59950
   0.59861
   0.60278
   0.60200
   0.60059
   0.60313
   0.60042
   0.60003
   0.59588
   0.59521
   0.60195
   0.59447
   0.60318
   0.59552
   0.59962

   0.60278
   0.60200
   0.60059
   0.60312
   0.60042
   0.60003
   0.59588
   0.59521
   0.60194
   0.59447
   0.60318
   0.59552
   0.59962
   0.59755
   0.60270
   0.60264
   0.59518
   0.59618
   0.60242
   0.59422
   0.59431
   0.59521
   0.59973
   0.59680
   0.60065
   0.59576
   0.59703
   0.60113
   0.59374
   0.59776
   0.59370
   0.59374
   0.60130
   0.60235
   0.59564
   0.60000
   0.60019

qs_dist =    2.6357e-06
Optimizing qs: remain iter - 4987
qs =

   0.60024
   0.59522
   0.59854
   0.59591
   0.59455
   0.59789
   0.60022
   0.59742
   0.60273
   0.60118
   0.60162
   0.59950
   0.59860
   0.60278
   0.60200
   0.60059
   0.60312
   0.60041
   0.60002
   0.59588
   0.59521
   0.60194
   0.59447
   0.60318
   0.59552
   0.59962
   0.59755
   0.60270
   0.60264
   0.59518
   0.59618
   0.60242
   0.59422
   0.59431
   0.59521
   0.59973
   0.59680
   0.60065
   0.59576
   0.59703
   0.60113
   0.59374
   0.59776
   0.59370
   0.59374
   0.60130
   0.60235
   0.59564


   0.59422
   0.59430
   0.59521
   0.59973
   0.59680
   0.60065
   0.59576

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.60312
   0.60041
   0.60002
   0.59588
   0.59521
   0.60194
   0.59447
   0.60317
   0.59552
   0.59962
   0.59754
   0.60269
   0.60264
   0.59517
   0.59618
   0.60242
   0.59422
   0.59430
   0.59520
   0.59973
   0.59680
   0.60065
   0.59576
   0.59703
   0.60112
   0.59373
   0.59776
   0.59370
   0.59372
   0.60130
   0.60235
   0.59563
   0.60000
   0.60019

qs_dist =    1.4801e-06
Optimizing qs: remain iter - 4961
qs =

   0.60024
   0.59522
   0.59853
   0.59591
   0.59455
   0.59788
   0.60022
   0.59742
   0.60273
   0.60117
   0.60162
   0.59950
   0.59859
   0.60278
   0.60200
   0.60058
   0.60312
   0.60041
   0.60002
   0.59588
   0.59521
   0.60194
   0.59447
   0.60317
   0.59552
   0.59962
   0.59754
   0.60269
   0.60264
   0.59517
   0.59618
   0.60242
   0.59422
   0.59430
   0.59520
   0.59973
   0.59680
   0.60065
   0.59576
   0.59703
   0.60112
   0.59373
   0.59776
   0.59370
   0.59372
   0.60130
   0.60235
   0.59563
   0.60000
   0.60019

qs_dist =

...

   0.59711
   0.59268
   0.59222
   0.60116
   0.60201
   0.59529
   0.59923
   0.59949

qs_dist =    1.0138e-06
Optimizing qs: remain iter - 211
qs =

   0.60024
   0.59522
   0.59810
   0.59524
   0.59424
   0.59765
   0.59963
   0.59657
   0.60225
   0.60041
   0.60073
   0.59923
   0.59810
   0.60242
   0.60175
   0.60041
   0.60203
   0.59999
   0.59978
   0.59510
   0.59487
   0.60156
   0.59388
   0.60205
   0.59493
   0.59889
   0.59692
   0.60208
   0.60197
   0.59468
   0.59537
   0.60161
   0.59264
   0.59361
   0.59479
   0.59955
   0.59601
   0.60008
   0.59478
   0.59641
   0.60019
   0.59312
   0.59711
   0.59267
   0.59222
   0.60116
   0.60201
   0.59529
   0.59923
   0.59949

qs_dist =    1.7346e-06
Optimizing qs: remain iter - 210
qs =

   0.60024
   0.59522
   0.59810
   0.59691
   0.60208
   0.60197
   0.59468
   0.59537
   0.60161
   0.59263
   0.59361
   0.59479
   0.59955
   0.59601
   0.60008
   0.59478
   0.59641
   0.60018
   0.59312
   0.59711
   0.59267
   

   0.59262


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.59886
   0.59691
   0.60207
   0.60195
   0.59467
   0.59535
   0.60158
   0.59260
   0.59360
   0.59478
   0.59954
   0.59599
   0.60007
   0.59474
   0.59641
   0.60017
   0.59310
   0.59710
   0.59265
   0.59218
   0.60115
   0.60200
   0.59528
   0.59921
   0.59947

qs_dist =    7.7216e-07
Optimizing qs: remain iter - 97
qs =

   0.60024
   0.59522
   0.59809
   0.59523
   0.59423
   0.59764
   0.59961
   0.59654
   0.60224
   0.60039
   0.60071
   0.59922
   0.59808
   0.60241
   0.60175
   0.60041
   0.60199
   0.59998
   0.59978
   0.59509
   0.59486
   0.60155
   0.59387
   0.60202
   0.59491
   0.59886
   0.59691
   0.60207
   0.60195
   0.59467
   0.59535
   0.60158
   0.59260
   0.59360
   0.59478
   0.59954
   0.59599
   0.60007
   0.59474
   0.59641
   0.60017
   0.59310
   0.59710
   0.59265
   0.59218
   0.60115
   0.60200
   0.59528
   0.59921
   0.59998
   0.59978
   0.59509
   0.59486
   0.60155
   0.59387
   0.60202
   0.59490
   0.59886
   0.59690
   0.60207
 

   0.59473
   0.59640
   0.60016
   0.59310
   0.59709
   0.59264
   0.59216
   0.60115
   0.60200
   0.59527
   0.59920
   0.59947

qs_dist =    5.0370e-07
Optimizing qs: remain iter - 44
qs =

   0.60024
   0.59522
   0.59809
   0.59522
   0.59423
   0.59764
   0.59961
   0.59653
   0.60224
   0.60039
   0.60070
   0.59921
   0.59808
   0.60241
   0.60174
   0.60040
   0.60199
   0.59998
   0.59977
   0.59508
   0.59486
   0.60155
   0.59387
   0.60201
   0.59490
   0.59886
   0.59690
   0.60206
   0.60194
   0.59467
   0.59534
   0.60157
   0.59258
   0.59358
   0.59477
   0.59954
   0.59598
   0.60007
   0.59473
   0.59640
   0.60016
   0.59310
   0.59709
   0.59264
   0.59216
   0.60115
   0.60200
   0.59527
   0.59920
   0.59947

qs_dist =    5.7959e-06
Optimizing qs: remain iter - 43
qs =

   0.60024
   0.59522
   0.59809
   0.59522
   0.59423
   0.59764
   0.59961
   0.59653
   0.60198
   0.59998
   0.59977
   0.59508
   0.59486
   0.60155
   0.59386
   0.60201
   0.59490
   0.

In [38]:
qs
save('./res/tegrbcmenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.60024
   0.59522
   0.59809
   0.59522
   0.59422
   0.59764
   0.59960
   0.59652
   0.60224
   0.60038
   0.60069
   0.59921
   0.59807
   0.60240
   0.60174
   0.60040
   0.60198
   0.59998
   0.59977
   0.59508
   0.59486
   0.60154
   0.59386
   0.60200
   0.59490
   0.59886
   0.59690
   0.60206
   0.60194
   0.59465
   0.59534
   0.60157
   0.59257
   0.59357
   0.59477
   0.59953
   0.59598
   0.60005
   0.59473
   0.59640
   0.60015
   0.59309
   0.59709
   0.59264
   0.59214
   0.60115
   0.60199
   0.59527
   0.59919
   0.59947

TEGRBCM: SMSE 0.00190003, MSLL -4.5521, NLPD -1.6739


## 1.3 GPoE vs TEGPoE

### 1.3.1 GPoE baseline

In [22]:
criterions = {'GPoE'}; %, 'GPoE', 'KLRBCM', 'KLGRBCM', 'GRBCM'

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion) ;
    [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
    printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);
    filename = sprintf('./res/%s_%s_%s.mat', dataset, partitionCriterion, criterion);
    save(filename, 'SMSE','MSLL','NLPD')
endfor

Processing energy_data kmeans GPoE
GPoE: SMSE 0.00147084, MSLL 2.9752, NLPD 5.8534


### 1.3.2 Grid q

In [23]:
partitionCriterions = {'kmeans'}; % 'random', 'kmeans', 'knkmeans'
criterions = {'TEGPoE'}; %, 'TEGRBCM' 'TEGPoE'};
qs = 0.2:0.2:2.0;

for j = 1:length(criterions)     
    criterion = criterions{j};
    printf('Processing %s %s %s\r\n', dataset, partitionCriterion, criterion);
    opts.partitionCriterion = partitionCriterion ;
    %[models,t_dGP_train] = aggregation_train(x,y,opts) ;
    for q = qs
        [mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
        [SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
        printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f - q %6.2f\r\n', criterion, SMSE,MSLL,NLPD,q);
        filename = sprintf('./res/tegpoe%s_%s_%s_%6.2f.mat', dataset, partitionCriterion, criterion, q);
        save(filename, 'SMSE','MSLL','NLPD')
    endfor
endfor

Processing energy_data kmeans TEGPoE
TEGPoE: SMSE 0.00175034, MSLL 5.1780, NLPD 8.0562 - q   0.20
TEGPoE: SMSE 0.00169386, MSLL 2.5783, NLPD 5.4565 - q   0.40
TEGPoE: SMSE 0.00162683, MSLL 1.9013, NLPD 4.7795 - q   0.60
TEGPoE: SMSE 0.00155116, MSLL 2.0704, NLPD 4.9486 - q   0.80
TEGPoE: SMSE 0.00147084, MSLL 2.9752, NLPD 5.8534 - q   1.00
TEGPoE: SMSE 0.00139096, MSLL 4.8186, NLPD 7.6968 - q   1.20
TEGPoE: SMSE 0.00131608, MSLL 8.0933, NLPD 10.9715 - q   1.40
TEGPoE: SMSE 0.00124901, MSLL 13.7279, NLPD 16.6061 - q   1.60
TEGPoE: SMSE 0.00119063, MSLL 23.4087, NLPD 26.2869 - q   1.80
TEGPoE: SMSE 0.00114052, MSLL 40.2017, NLPD 43.0799 - q   2.00


### 1.3.3 Find single q

In [24]:
criterion = 'TEGPoE';

In [25]:
printf('Optimizing q\r\n');
q = grad_q(models, criterion, opts, iter=1000, lr=1e-9, lambda=1e6, init_q=0.6);

Optimizing q
Msize =

 Columns 1 through 13:

   360   194   469   503   333   469   571   424   181   502   497   338   320

 Columns 14 through 26:

   354   277   174   314   404   340   617   339   121   411   253   265   360

 Columns 27 through 39:

   287   323   366   141   510   505   427   339   292   345   441   319   436

 Columns 40 through 50:

   638   194   378   378   500   376   361   185   163   204   502

Start preprocessing...
Finish preprocessing...
Optimizing q: remain iter - 1000, q - 0.6000
grad_q_norm = -12.415
grad_q_reg = -43.948
Optimizing q: remain iter - 999, q - 0.6009
grad_q_norm =  75.466
grad_q_reg = -99.046
Optimizing q: remain iter - 998, q - 0.6029
grad_q_norm =  53.812
grad_q_reg = -89.755
Optimizing q: remain iter - 997, q - 0.6047
grad_q_norm =  53.295
grad_q_reg = -72.952
Optimizing q: remain iter - 996, q - 0.6062
grad_q_norm =  58.291
grad_q_reg = -81.649
Optimizing q: remain iter - 995, q - 0.6079
grad_q_norm =  75.293
grad_q_reg = -91.482
O

grad_q_norm =  10.100
grad_q_reg = -36.316
Optimizing q: remain iter - 911, q - 0.6991
grad_q_norm =  26.246
grad_q_reg = -39.925
Optimizing q: remain iter - 910, q - 0.6999
grad_q_norm = -11.755
grad_q_reg =  0.13376
Optimizing q: remain iter - 909, q - 0.6999
grad_q_norm =  16.750
grad_q_reg = -41.964
Optimizing q: remain iter - 908, q - 0.7008
grad_q_norm =  30.362
grad_q_reg = -36.843
Optimizing q: remain iter - 907, q - 0.7015
grad_q_norm =  39.945
grad_q_reg = -53.017
Optimizing q: remain iter - 906, q - 0.7026
grad_q_norm = -0.28624
grad_q_reg = -11.625
Optimizing q: remain iter - 905, q - 0.7028
grad_q_norm =  26.651
grad_q_reg = -34.892
Optimizing q: remain iter - 904, q - 0.7035
grad_q_norm =  9.3373
grad_q_reg = -17.178
Optimizing q: remain iter - 903, q - 0.7039
grad_q_norm =  34.037
grad_q_reg = -48.300
Optimizing q: remain iter - 902, q - 0.7049
grad_q_norm =  10.334
grad_q_reg = -21.765
Optimizing q: remain iter - 901, q - 0.7053
grad_q_norm = -15.095
grad_q_reg =  1.763

Optimizing q: remain iter - 818, q - 0.7451
grad_q_norm = -10.415
grad_q_reg = -17.108
Optimizing q: remain iter - 817, q - 0.7455
grad_q_norm = -6.1268
grad_q_reg = -24.433
Optimizing q: remain iter - 816, q - 0.7460
grad_q_norm = -46.746
grad_q_reg = -42.625
Optimizing q: remain iter - 815, q - 0.7468
grad_q_norm = -48.288
grad_q_reg = -41.679
Optimizing q: remain iter - 814, q - 0.7477
grad_q_norm = -21.545
grad_q_reg =  15.043
Optimizing q: remain iter - 813, q - 0.7474
grad_q_norm =  7.1157
grad_q_reg = -17.996
Optimizing q: remain iter - 812, q - 0.7477
grad_q_norm =  51.439
grad_q_reg = -56.915
Optimizing q: remain iter - 811, q - 0.7489
grad_q_norm =  0.047615
grad_q_reg = -14.023
Optimizing q: remain iter - 810, q - 0.7492
grad_q_norm = -69.302
grad_q_reg = -13.251
Optimizing q: remain iter - 809, q - 0.7495
grad_q_norm = -28.507
grad_q_reg =  14.367
Optimizing q: remain iter - 808, q - 0.7492
grad_q_norm = -17.567
grad_q_reg =  10.681
Optimizing q: remain iter - 807, q - 0.74

Optimizing q: remain iter - 724, q - 0.7634
grad_q_norm =  7.0439
grad_q_reg = -2.9737
Optimizing q: remain iter - 723, q - 0.7634
grad_q_norm = -26.674
grad_q_reg =  19.633
Optimizing q: remain iter - 722, q - 0.7630
grad_q_norm =  10.834
grad_q_reg = -17.136
Optimizing q: remain iter - 721, q - 0.7634
grad_q_norm =  10.534
grad_q_reg = -34.237
Optimizing q: remain iter - 720, q - 0.7641
grad_q_norm = -21.675
grad_q_reg =  7.8597
Optimizing q: remain iter - 719, q - 0.7639
grad_q_norm =  14.220
grad_q_reg = -28.832
Optimizing q: remain iter - 718, q - 0.7645
grad_q_norm =  7.9092
grad_q_reg = -12.368
Optimizing q: remain iter - 717, q - 0.7647
grad_q_norm = -32.088
grad_q_reg = -28.998
Optimizing q: remain iter - 716, q - 0.7653
grad_q_norm = -39.679
grad_q_reg =  27.139
Optimizing q: remain iter - 715, q - 0.7648
grad_q_norm = -17.279
grad_q_reg = -23.907
Optimizing q: remain iter - 714, q - 0.7653
grad_q_norm = -11.834
grad_q_reg =  2.9639
Optimizing q: remain iter - 713, q - 0.7652

Optimizing q: remain iter - 630, q - 0.7702
grad_q_norm = -39.946
grad_q_reg =  34.049
Optimizing q: remain iter - 629, q - 0.7695
grad_q_norm = -85.716
grad_q_reg = -16.058
Optimizing q: remain iter - 628, q - 0.7698
grad_q_norm = -17.027
grad_q_reg =  2.5971
Optimizing q: remain iter - 627, q - 0.7697
grad_q_norm = -7.4970
grad_q_reg = -5.4772


...


Optimizing q: remain iter - 469, q - 0.7769
grad_q_norm = -41.916
grad_q_reg =  27.838
Optimizing q: remain iter - 468, q - 0.7764
grad_q_norm = -41.752
grad_q_reg =  46.492
Optimizing q: remain iter - 467, q - 0.7754
grad_q_norm = -35.875
grad_q_reg =  25.708
Optimizing q: remain iter - 466, q - 0.7749
grad_q_norm = -21.276
grad_q_reg =  5.5747
Optimizing q: remain iter - 465, q - 0.7748
grad_q_norm = -35.697
grad_q_reg = -3.5406
Optimizing q: remain iter - 464, q - 0.7749
grad_q_norm = -36.391
grad_q_reg =  25.057
Optimizing q: remain iter - 463, q - 0.7743
grad_q_norm =  40.975
grad_q_reg = -53.233
Optimizing q: remain iter - 462, q - 0.7754
grad_q_norm =  10.549
grad_q_reg = -15.595
Optimizing q: remain iter - 461, q - 0.7757
grad_q_norm = -52.334
grad_q_reg = -27.780
Optimizing q: remain iter - 460, q - 0.7763
grad_q_norm =  1.1818
grad_q_reg = -26.016
Optimizing q: remain iter - 459, q - 0.7768
grad_q_norm = -50.942
grad_q_reg =  36.268
Optimizing q: remain iter - 458, q - 0.776

Optimizing q: remain iter - 375, q - 0.7796
grad_q_norm = -4.5850
grad_q_reg = -11.753
Optimizing q: remain iter - 374, q - 0.7799
grad_q_norm =  0.23666
grad_q_reg = -24.488
Optimizing q: remain iter - 373, q - 0.7804
grad_q_norm =  7.1946
grad_q_reg = -16.219
Optimizing q: remain iter - 372, q - 0.7807
grad_q_norm = -40.134
grad_q_reg =  21.954
Optimizing q: remain iter - 371, q - 0.7803
grad_q_norm = -35.425
grad_q_reg =  31.309
Optimizing q: remain iter - 370, q - 0.7796
grad_q_norm = -20.958
grad_q_reg = -14.556
Optimizing q: remain iter - 369, q - 0.7799
grad_q_norm =  16.001
grad_q_reg = -35.922
Optimizing q: remain iter - 368, q - 0.7806
grad_q_norm = -18.482
grad_q_reg =  10.349
Optimizing q: remain iter - 367, q - 0.7804
grad_q_norm = -28.491
grad_q_reg = -10.821
Optimizing q: remain iter - 366, q - 0.7807
grad_q_norm = -43.784
grad_q_reg =  35.345
Optimizing q: remain iter - 365, q - 0.7799
grad_q_norm =  9.5923
grad_q_reg = -23.863
Optimizing q: remain iter - 364, q - 0.780

Optimizing q: remain iter - 281, q - 0.7786
grad_q_norm = -24.489
grad_q_reg = -3.0360
Optimizing q: remain iter - 280, q - 0.7787
grad_q_norm = -29.622
grad_q_reg =  21.312
Optimizing q: remain iter - 279, q - 0.7783
grad_q_norm = -15.010
grad_q_reg =  4.9708
Optimizing q: remain iter - 278, q - 0.7782
grad_q_norm = -27.987
grad_q_reg =  20.474
Optimizing q: remain iter - 277, q - 0.7777
grad_q_norm = -36.569
grad_q_reg =  44.960
Optimizing q: remain iter - 276, q - 0.7768
grad_q_norm = -21.163
grad_q_reg =  12.521
Optimizing q: remain iter - 275, q - 0.7766
grad_q_norm =  17.497
grad_q_reg = -34.180
Optimizing q: remain iter - 274, q - 0.7773
grad_q_norm =  8.8996
grad_q_reg = -12.328
Optimizing q: remain iter - 273, q - 0.7775
grad_q_norm =  43.600
grad_q_reg = -47.185
Optimizing q: remain iter - 272, q - 0.7785
grad_q_norm =  26.074
grad_q_reg = -25.782
Optimizing q: remain iter - 271, q - 0.7790
grad_q_norm = -35.626
grad_q_reg = -16.408
Optimizing q: remain iter - 270, q - 0.7793

Optimizing q: remain iter - 187, q - 0.7809
grad_q_norm = -73.314
grad_q_reg = -18.136
Optimizing q: remain iter - 186, q - 0.7812
grad_q_norm = -59.354
grad_q_reg =  40.312
Optimizing q: remain iter - 185, q - 0.7804
grad_q_norm = -42.486
grad_q_reg =  30.528
Optimizing q: remain iter - 184, q - 0.7798
grad_q_norm =  13.387
grad_q_reg = -9.6210
Optimizing q: remain iter - 183, q - 0.7800
grad_q_norm = -25.811
grad_q_reg =  16.516
Optimizing q: remain iter - 182, q - 0.7796
grad_q_norm = -65.612
grad_q_reg =  57.387
Optimizing q: remain iter - 181, q - 0.7785
grad_q_norm = -11.733
grad_q_reg =  0.96278
Optimizing q: remain iter - 180, q - 0.7784
grad_q_norm = -0.83440
grad_q_reg = -8.8127
Optimizing q: remain iter - 179, q - 0.7786
grad_q_norm = -37.299
grad_q_reg =  16.535
Optimizing q: remain iter - 178, q - 0.7783
grad_q_norm =  12.016
grad_q_reg = -52.356
Optimizing q: remain iter - 177, q - 0.7794
grad_q_norm = -22.052
grad_q_reg =  14.166
Optimizing q: remain iter - 176, q - 0.77

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [26]:
[mu_dGP,s2_dGP,t_dGP_predict] = aggregation_predict(xt,models,criterion,q) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

### 1.3.4 Find multi q

In [27]:
printf('Optimizing q\r\n');
iqs = q + 0.01* rand(opts.Ms,1);
qs = mul_grad_q(models, criterion, opts, iter=5000, lr=1e-8, lambda=1e6, init_q=iqs);
[mu_dGP,s2_dGP,t_dGP_predict] = mul_aggregation_predict(xt,models,criterion,qs) ;
[SMSE,MSLL,NLPD] = evaluate(x, y, xt, yt, mu_dGP, s2_dGP);

Optimizing q
Start preprocessing...
Finish preprocessing...
Optimizing qs: remain iter - 5000
qs =

   0.78235
   0.78630
   0.78670
   0.78279
   0.78893
   0.78378
   0.78814
   0.79020
   0.78981
   0.78933
   0.78665
   0.78963
   0.78148
   0.78351
   0.78971
   0.78526
   0.78825
   0.78931
   0.78623
   0.78875
   0.78755
   0.78442
   0.78870
   0.78159
   0.78607
   0.78978
   0.78144
   0.78790
   0.78428
   0.78414
   0.78725
   0.78149
   0.79032
   0.78695
   0.78528
   0.78291
   0.78876
   0.78320
   0.78562
   0.78304
   0.78999
   0.78705
   0.78372
   0.78206
   0.78165
   0.78842
   0.78437
   0.79092
   0.78940
   0.78224

qs_dist =    1.2690e-04
Optimizing qs: remain iter - 4999
qs =

   0.78235
   0.78631
   0.78673
   0.78281
   0.78893
   0.78378
   0.78814
   0.79020
   0.78982
   0.78936
   0.78668
   0.78963
   0.78150
   0.78351
   0.78971
   0.78526
   0.78825
   0.78931
   0.78623
   0.78875
   0.78755
   0.78442
   0.78871
   0.78160
   0.78607
   0.78981

   0.78982
   0.78935
   0.78680
   0.78967
   0.78161
   0.78336
   0.78973
   0.78537
   0.78791
   0.78908
   0.78619
   0.78856
   0.78744
   0.78452
   0.78864
   0.78151
   0.78595
   0.78989
   0.78163
   0.78797
   0.78448
   0.78426
   0.78689
   0.78143
   0.79038
   0.78707
   0.78547
   0.78312
   0.78850
   0.78322
   0.78523
   0.78289
   0.79015
   0.78712
   0.78384
   0.78230
   0.78173
   0.78863
   0.78449
   0.79106
   0.78930
   0.78231

qs_dist =    1.7959e-04
Optimizing qs: remain iter - 4986
qs =

   0.78125
   0.78657
   0.78692
   0.78276
   0.78904
   0.78400
   0.78779
   0.78994
   0.78982
   0.78938
   0.78682
   0.78967
   0.78163
   0.78336
   0.78974
   0.78537
   0.78791
   0.78908
   0.78619
   0.78856
   0.78744
   0.78452
   0.78865
   0.78151
   0.78595
   0.78992
   0.78165
   0.78798
   0.78449
   0.78429
   0.78689
   0.78143
   0.79022
   0.78710
   0.78548
   0.78313
   0.78850
   0.78322
   0.78523
   0.78290
   0.79017
   0.78713
   0.78386


   0.79002
   0.78172
   0.78809
   0.78449
   0.78443
   0.78675
   0.78161
   0.79035
   0.78698
   0.78553
   0.78316
   0.78857
   0.78324

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.78771
   0.78481
   0.78881
   0.78137
   0.78611
   0.79012
   0.78147
   0.78792
   0.78454
   0.78453
   0.78678
   0.78153
   0.79032
   0.78720
   0.78566
   0.78352
   0.78871
   0.78349
   0.78523
   0.78278
   0.79004
   0.78735
   0.78410
   0.78202
   0.78186
   0.78912
   0.78473
   0.79137
   0.78940
   0.78250

qs_dist =    2.0510e-04
Optimizing qs: remain iter - 4960
qs =

   0.77890
   0.78665
   0.78734
   0.78253
   0.78926
   0.78452
   0.78793
   0.78991
   0.79002
   0.78904
   0.78640
   0.78978
   0.78181
   0.78374
   0.78988
   0.78570
   0.78806
   0.78936
   0.78630
   0.78833
   0.78773
   0.78481
   0.78883
   0.78140
   0.78611
   0.79012
   0.78150
   0.78795
   0.78454
   0.78448
   0.78679
   0.78156
   0.79027
   0.78721
   0.78569
   0.78354
   0.78872
   0.78351
   0.78524
   0.78280
   0.79005
   0.78737
   0.78411
   0.78196
   0.78182
   0.78913
   0.78475
   0.79140
   0.78940
   0.78252

qs_dist =    3.8535e-04
Optimizing qs: remain iter - 

...

   0.79665
   0.71642
   0.77989
   0.82409
   0.75928
   0.73201
   0.76429
   0.82438
   0.77965
   0.80616
   0.81427
   0.82860
   0.78053
   0.74634
   0.80273
   0.82259
   0.82985
   0.85349
   0.74936
   0.79078
   0.74559
   0.74744
   0.79547
   0.80247
   0.82472
   0.78527
   0.80140
   0.86259
   0.79889
   0.84249
   0.83454
   0.76425

qs_dist =    2.6064e-04
Optimizing qs: remain iter - 203
qs =

   0.61376
   0.81197
   0.84679
   0.76561
   0.81569
   0.84480
   0.74503
   0.72168
   0.77012
   0.78210
   0.77292
   0.80747
   0.80353
   0.79418
   0.80309
   0.82571
   0.78715
   0.79571
   0.79666
   0.71643
   0.77992
   0.82412
   0.75930
   0.73205
   0.76432
   0.82439
   0.77960
   0.80615
   0.81429
   0.82861
   0.78055
   0.74638
   0.80274
   0.82261
   0.82986
   0.85351
   0.74940
   0.79081
   0.74562
   0.74740
   0.79548
   0.80249
   0.82474
   0.78530
   0.80140
   0.86261
   0.79891
   0.84251
   0.83456
   0.76428

qs_dist =    2.9798e-04
Optimizin

   0.78172
   0.77259
   0.80787
   0.80379
   0.79439


IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)




   0.84795
   0.76534
   0.81624
   0.84607
   0.74421
   0.72070
   0.76956
   0.78165
   0.77261
   0.80785
   0.80400
   0.79442
   0.80352
   0.82594
   0.78728
   0.79559
   0.79630
   0.71541
   0.77963
   0.82499
   0.75901
   0.73102
   0.76393
   0.82539
   0.77916
   0.80634
   0.81475
   0.82971
   0.78038
   0.74601
   0.80300
   0.82313
   0.83097
   0.85519
   0.74813
   0.79143
   0.74501
   0.74643
   0.79556
   0.80275
   0.82550
   0.78507
   0.80177
   0.86433
   0.79920
   0.84344
   0.83552
   0.76371

qs_dist =    2.9793e-04
Optimizing qs: remain iter - 81
qs =

   0.61266
   0.81218
   0.84797
   0.76530
   0.81626
   0.84610
   0.74421
   0.72065
   0.76955
   0.78163
   0.77261
   0.80785
   0.80400
   0.79441
   0.80352
   0.82596
   0.78730
   0.79558
   0.79615
   0.71531
   0.77961
   0.82501
   0.75899
   0.73095
   0.76387
   0.82540
   0.77913
   0.80636
   0.81477
   0.82973
   0.78039
   0.74597
   0.80303
   0.82314
   0.83095
   0.85522
   0.74803
 

   0.76989
   0.78177
   0.77294
   0.80816
   0.80435
   0.79450
   0.80364
   0.82630
   0.78716
   0.79579
   0.79648
   0.71554
   0.77985
   0.82571
   0.75922
   0.73068
   0.76395
   0.82563
   0.77907
   0.80633
   0.81475
   0.82995
   0.78018
   0.74607
   0.80264
   0.82309
   0.83164
   0.85595
   0.74797
   0.79183
   0.74479
   0.74661
   0.79549
   0.80298
   0.82541
   0.78480
   0.80187
   0.86499
   0.79933
   0.84394
   0.83567
   0.76369

qs_dist =    3.0921e-04
Optimizing qs: remain iter - 27
qs =

   0.61251
   0.81229
   0.84827
   0.76528
   0.81704
   0.84669
   0.74451
   0.72063
   0.76990
   0.78163
   0.77280
   0.80819
   0.80436
   0.79453
   0.80364
   0.82632
   0.78718
   0.79582
   0.79650
   0.71554
   0.77988
   0.82573
   0.75922
   0.73066
   0.76398
   0.82566
   0.77900
   0.80628
   0.81471
   0.82993
   0.78020
   0.74608
   0.80262
   0.82311
   0.83160
   0.85598
   0.74798
   0.79185
   0.74483
   0.74655
   0.79540
   0.80299
   0.82544
  

In [28]:
qs
save('./res/tegpoeenergydataqs', 'qs');
printf('%s: SMSE %6.8f, MSLL %6.4f, NLPD %6.4f\r\n', criterion, SMSE,MSLL,NLPD);

qs =

   0.61244
   0.81220
   0.84868
   0.76556
   0.81721
   0.84701
   0.74454
   0.72065
   0.77004
   0.78155
   0.77256
   0.80825
   0.80445
   0.79485
   0.80371
   0.82655
   0.78718
   0.79611
   0.79691
   0.71554
   0.78013
   0.82603
   0.75948
   0.73064
   0.76407
   0.82580
   0.77913
   0.80656
   0.81455
   0.83007
   0.78030
   0.74641
   0.80286
   0.82311
   0.83139
   0.85608
   0.74798
   0.79215
   0.74506
   0.74653
   0.79538
   0.80318
   0.82553
   0.78386
   0.80221
   0.86538
   0.79958
   0.84432
   0.83603
   0.76374

TEGPoE: SMSE 0.00154981, MSLL 1.9612, NLPD 4.8394
